<a href="https://colab.research.google.com/github/trivedidn/BNFO301_Trivedi_Disha/blob/main/Lab5_Matrix_Disha_Trivedi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>BNFO301 Homework Assignment #3: Consensus Sequences</h1>

A consensus sequence represents the most frequent nucleotide at each position in an alignment of two or more sequences. You can think of this as finding the most common base in a position within a multiple sequence alignment. Consensus sequences can be useful for identifying and visualizing motifs in a DNA or amino acid sequence. You have been provided a FASTA file of aligned sequences. The goal of this assignment is to write a python script to read in the sequences from the file and generate the consensus sequence




**Helpful Resources:**

If you are unfamiliar with consensus sequences or would like to review, this video explains the concept: https://www.youtube.com/watch?v=4HYJILahPw4 (2:20 to 4:20 most useful). It may also be helpful to first generate the consensus sequence by hand, so that you can check the output from your script.

Grading: attendance = 4 points (2 points for each meeting); problem 1 = 1 point; problem 2 = 3 points; problem 3 = 2 points.  There is a bonus question at the end worth 2 points.

The assignment is to be completed in class, not outside of class.  You will have two class periods to complete the lab.  The bonus may be completed on your own between meetings.

### Setup
Load the Sequence Data File.  Please run this block without changing the code.

File : https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta

In [1]:
import os.path
# Load the genbank file 
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta"
DEFAULT_FILE_NAME = 'Assignment3_data.fasta'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB
   


--2022-03-03 17:40:16--  https://raw.githubusercontent.com/MusBansal/BNFO301Data/main/Assignment3_data.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 312 [text/plain]
Saving to: ‘Assignment3_data.fasta’

Assignment3_data.fa 100%[===================>]     312  --.-KB/s    in 0s      

2022-03-03 17:40:17 (8.64 MB/s) - ‘Assignment3_data.fasta’ saved [312/312]



###Step-1.  Create a definition to clean the FASTA formated file and add sequences into a list.  Create a second list that contains your headers (with the ">" removed).  Your function should return both lists, not just print each list.  Finally, I suggest you create function that can print the each list.  This later function will allow you to evaluate if your dictionary contains the expected information.


Hint, you will need to do the following:
* We need a function to load the sequence file
* We will load it in two lists of sequences and header names
* Remember to strip out ">" and "\n"


Your output should look like this:

Human       GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Gorilla     GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Dolphin     GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG

Rat         GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC

Mouse       GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


In [2]:
#-----------------------------------------
#Read in fasta and create list of sequences (sequences) and a list of sequence identifiers (seqIDs)
#-----------------------------------------
def ReadFile(fh):
    file = open(fh)

    # Initialize variables
    sequences = []  # Store sequences
    seqIds = []  # Store sequence headers
    
    for lines in open(fh):
      lines = lines.strip() 
      if lines.startswith(">"): 
        lines = lines.strip(">")
        seqIds.append(lines)
       
      else:
        sequences.append(lines)
        


    return sequences, seqIds



# -----------------------------
# Create utility function to print sequences and seqIDs in an easily readable format 
# ------------------------------

sequences, seqIds = ReadFile(fileName)

print("Sequences:", sequences)
print("Seq Ids:", seqIds)



Sequences: ['GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT', 'GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT', 'GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG', 'GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC', 'GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC']
Seq Ids: [' Human', ' Gorilla', ' Dolphin', ' Rat', ' Mouse']


###Step-2. In this step you will build a rutine that uses nested loops to format the data in such a way that you can determine the most common base.  This is very similar to the process we discussed in lecture on pairwise alignment.  Remember to return your data.  Finally, print the first three columns of the matrix.  This you can visualize your matrix and check it for accuracy.  Functions len and range will be useful in completing this problem.


For example, lets take the input for the first three bases in the alignment given:

Human       GGA

Gorilla     GGA

Dolphin     GAG

Rat         GGA

Mouse       GAG

You should return data that looks like this:  [['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G']]

In [6]:
# -----------------------------
# Convert sequence format for easy computation
# ------------------------------
def maniputlateSeqs(seqs):
    # Lengths of the aligned sequences are equal
    # Get length of first sequence
    length = len(seqs[0])
    count = 0 
    # Initialize variables
    NewlyFormatedData = []  # This will store all values at each position
  
    # Create a list of lists using nested loops
    for base in range (length):
        column = []
        for sequences in range(len(seqs)): 
          column.append(seqs[sequences][base])
        NewlyFormatedData.append(column)

    # Reads fasta file as matrix
    # Example:
    #        ATGCA
    #        ATGAA
    #        TCGAT
    #             Bases at index 0   Bases at index 1 ...
    # positions = [["A", "A", "T"],  ["T", "T", "C"], ...]]



    
    
    
    
    return NewlyFormatedData



#print the first 5 rows
Output = maniputlateSeqs(sequences)
print(Output[0:5])



[['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G'], ['G', 'G', 'G', 'G', 'G'], ['A', 'A', 'C', 'C', 'C']]


###Step-3.  In this last step, you will create a function that returns a consensus sequence using the most fequent base. You will again need to use nested loops.  Functions get and max will be helpful in responding to this question.  

Use the following rules:  This function should print the concensus sequence as one continuous string. 

1. If all bases are in agreement, return base as upper.case.  e.g. ['G', 'G', 'G', 'G', 'G'] returns G
2. If there is an equal split between bases in a column, return "n".  e.g. ['G', 'G', 'A', 'A', 'A', 'G'] returns n
3. If the the bases are not in agreement, but one base is more frequent than other, return the most frequent base in lower.case.  eg. ['G', 'G', 'G', 'G', 'A'] returns g

Your output should look like this: GgaGcg?ct?GGAgCCgGacCcgGAcCcCgGCGAt?GCcGcCtGc?tCtC?


In [10]:
def getConsensus(NewlyFormatedData):
    # Initialize variables
    consensus = ""
    keys = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    base_length = len(NewlyFormatedData[0])

    # Use nested loop through each inner list
    for i in range(base_length):
      max_freq = -1
      max_freq_base = 0 
      for base in 'ATGC': 
        if NewlyFormatedData[keys][base][i] > max_freq: # See if there are multiple keys with the max value
          max_freq = NewlyFormatedData[keys][base][i] # Calculate abundance of base at position
          max_freq_base = base 
        elif NewlyFormatedData[keys][base][i] == max_freq: 
          max_freq_base = "n"

      consensus += max_freq_base
    
        

        
        

        
        

    return consensus


#print consensus
    Output = getConsensus(consensus)
    print(consensus)


###Bonus  
One way aligned sequences are often visualized is as a sequence logo. For an additional point, go to https://weblogo.berkeley.edu/logo.cgi and upload the provided FASTA file. Save the image and upload to canvas.  You may do the bonus between class meetings on your own time.